In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup
import time
import json
import pandas as pd


In [2]:
options = Options()
options.headless = False 
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
options.add_argument("disable-infobars")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

### Mantel


In [4]:
url = "https://www.zara.com/de/de/damen-jacken-maentel-l1184.html?v1=2419032"

driver.get(url)
time.sleep(5)

wait = WebDriverWait(driver, 10)

time.sleep(3)

def gradual_scroll(driver, pause_time, step_size):
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        for i in range(0, last_height, step_size):
            driver.execute_script(f"window.scrollTo(0, {i});")
            time.sleep(pause_time)
        
        time.sleep(pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        last_height = new_height

# Hacer scroll gradual con pausas y pasos pequeños
gradual_scroll(driver, pause_time=0.2, step_size=50)

# Espera adicional para asegurarse de que todo el contenido ha cargado
time.sleep(5)

# Extraer el HTML de la página
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

# Cerrar el navegador
driver.quit()

# Guardar el HTML en un archivo para análisis posterior
#with open("zara_mujer_blazer.html", "w", encoding='utf-8') as file:
    #file.write(page_source)

print("El scroll, el cambio de vista y la extracción han sido completados.")

products = soup.find_all('li', class_='product-grid-product')

category = "neuware"
sex = "damen"


data = {
    "Category":[],
    "Sex": [], 
    "Product": [],
    "Color": [],
    "Subcategory": [],
    "Reference": [],
    "Price": [],
    "Image": [],
    "Link": []
}

for product in products:
    # Extraer el nombre y color del producto desde el atributo 'alt' de <img>
    image_tag = product.find('img', class_='media-image__image')
    if image_tag and 'alt' in image_tag.attrs:
        alt_text = image_tag['alt']
        # Verificar si se puede dividir en dos partes: nombre y color
        split_text = alt_text.split(' - ')
        if len(split_text) == 2:
            name, color = split_text
        else:
            name = alt_text  # Si no hay un guion, tomamos todo como el nombre
            color = "No Color"  # Color por defecto
    else:
        name = "No Name"
        color = "No Color"
    
    # Extraer el enlace al producto desde el <a> con clase 'product-link'
    link = product.find('a', class_='product-link')['href'] if product.find('a', class_='product-link') else "No Link"
    
    # Extraer la subcategoría desde el link, basándonos en palabras clave
    subcategory = link.split('/')[-1].split('-')[0]  # Primera palabra de la URL del producto (ej. "capa")

    # Extraer el precio del producto desde el <span> con clase 'money-amount__main'
    price = product.find('span', class_='money-amount__main').get_text(strip=True) if product.find('span', class_='money-amount__main') else "No Price"
    
    # Extraer la referencia del producto (viene como 'data-productid' en la etiqueta <li>)
    reference = product.get('data-productid', 'No Reference')
    
    # Extraer la imagen del producto desde el <img> con clase 'media-image__image'
    image = product.find('img', class_='media-image__image')['src'] if product.find('img', class_='media-image__image') else "No Image"
    
    #category = category

    #sex = sex 

    # Guardar la información en el diccionario
    data["Category"].append(category)
    data["Sex"].append(sex)
    data["Product"].append(name)
    data["Color"].append(color)
    data["Subcategory"].append(subcategory)
    data["Reference"].append(reference)
    data["Price"].append(price)
    data["Image"].append(image)
    data["Link"].append(link)

# Convertir el diccionario en un DataFrame de pandas
df = pd.DataFrame(data)

df.to_csv("../01_data/df_zara_mantel.csv", index= True)

In [102]:
#df.to_csv("../01_data/df_zara_mantel.csv", index= True)

In [3]:
options = Options()
options.headless = False 
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
options.add_argument("disable-infobars")


def scrape_zara_data(urls, categories):
    # Initialize the driver (Make sure to set up your driver path accordingly)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)  # Set the path to your ChromeDriver

    # Data storage
    data = {
        "Category": [],
        "Sex": [],
        "Product": [],
        "Color": [],
        "Subcategory": [],
        "Reference": [],
        "Price": [],
        "Image": [],
        "Link": []
    }

    # Iterate over both URLs and Categories
    for url, category in zip(urls, categories):
        # Open the URL
        driver.get(url)
        time.sleep(5)

        # Scroll the page gradually
        def gradual_scroll(driver, pause_time, step_size):
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                for i in range(0, last_height, step_size):
                    driver.execute_script(f"window.scrollTo(0, {i});")
                    time.sleep(pause_time)
                time.sleep(pause_time)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

        gradual_scroll(driver, pause_time=0.2, step_size=50)
        
        # Additional wait to ensure content loads
        time.sleep(5)

        # Extract the HTML of the page
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # Extract product data
        products = soup.find_all('li', class_='product-grid-product')

        sex = "damen"  # Assuming sex is constant in this case, you can modify it as needed

        for product in products:
            image_tag = product.find('img', class_='media-image__image')
            if image_tag and 'alt' in image_tag.attrs:
                alt_text = image_tag['alt']
                split_text = alt_text.split(' - ')
                if len(split_text) == 2:
                    name, color = split_text
                else:
                    name = alt_text
                    color = "No Color"
            else:
                name = "No Name"
                color = "No Color"

            link = product.find('a', class_='product-link')['href'] if product.find('a', class_='product-link') else "No Link"
            subcategory = link.split('/')[-1].split('-')[0]
            price = product.find('span', class_='money-amount__main').get_text(strip=True) if product.find('span', class_='money-amount__main') else "No Price"
            reference = product.get('data-productid', 'No Reference')
            image = product.find('img', class_='media-image__image')['src'] if product.find('img', class_='media-image__image') else "No Image"

            data["Category"].append(category)  # Use the dynamic category
            data["Sex"].append(sex)
            data["Product"].append(name)
            data["Color"].append(color)
            data["Subcategory"].append(subcategory)
            data["Reference"].append(reference)
            data["Price"].append(price)
            data["Image"].append(image)
            data["Link"].append(link)

    # Close the browser
    driver.quit()

    # Convert the data to a pandas DataFrame
    df_zara = pd.DataFrame(data)
    
    # Save the DataFrame as a CSV file
    df_zara.to_csv("../01_data/df_zara.csv", index=True)

    print("Scraping completed and data saved to CSV.")

# Example usage with multiple URLs and corresponding categories:
urls = [
    "https://www.zara.com/de/de/damen-jacken-maentel-l1184.html?v1=2419032",
    "https://www.zara.com/de/de/damen-jacken-l1114.html?v1=2417772",
    "https://www.zara.com/de/de/damen-blazer-l1055.html?v1=2420942",
    "https://www.zara.com/de/de/damen-kleider-l1066.html?v1=2420896",
    "https://www.zara.com/de/de/damen-hosen-l1335.html?v1=2420795",
    "https://www.zara.com/de/de/damen-t-shirts-l1362.html?v1=2420417"
    "https://www.zara.com/de/de/damen-roecke-l1299.html?v1=2420454",
    "https://www.zara.com/de/de/damen-schuhe-l1251.html?v1=2419160",
    "https://www.zara.com/de/de/damen-taschen-l1024.html?v1=2417728",
    "https://www.zara.com/de/de/damen-lingerie-l4021.html?v1=2419807",
    "https://www.zara.com/de/de/damen-schonheit-parfums-l1415.html?v1=2419833",
    "https://www.zara.com/de/de/woman-beauty-makeup-l4414.html?v1=2418919"

    # Add more URLs as needed
]

categories = [
    "maentel",
    "jacken",
    "blazer",
    "kleider",
    "hosen",
    "t_shirts",
    "roecke",
    "schuhe",
    "taschen",
    "lingerie",
    "parfums",
    "make_up"


    # Add corresponding categories as needed
]

scrape_zara_data(urls, categories)

Scraping completed and data saved to CSV.


In [11]:
#df2= pd.read_csv("../01_data/df_zara.csv", index =False)

TypeError: read_csv() got an unexpected keyword argument 'index'

In [13]:
df2


,Unnamed: 0,Category,Sex,Product,Color,Subcategory,Reference,Price,Image,Link
0,0,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun / Braungrau von Zara,langer,410888372,"89,95",https://static.zara.net/assets/public/a9e5/f05...,https://www.zara.com/de/de/langer-wendemantel-...
1,1,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun / Braungrau von Zara,langer,410774601,"89,95",https://static.zara.net/assets/public/0a65/b2f...,https://www.zara.com/de/de/langer-wendemantel-...
2,2,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun / Braungrau von Zara,langer,410888372,"89,95",https://static.zara.net/assets/public/fe16/c55...,https://www.zara.com/de/de/langer-wendemantel-...
3,3,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun / Braungrau von Zara,langer,410888372,"89,95",https://static.zara.net/assets/public/868d/7ce...,https://www.zara.com/de/de/langer-wendemantel-...
4,4,maentel,damen,STRICKMANTEL MIT AUFGESETZTEN TASCHEN,Grau meliert von Zara,strickmantel,385398845,"69,95",https://static.zara.net/assets/public/0aa6/a50...,https://www.zara.com/de/de/strickmantel-mit-au...
...,...,...,...,...,...,...,...,...,...,...
5647,5647,parfums,damen,EYE GLOSS,von Zara,eye,393335942,"9,95",https://static.zara.net/assets/public/b847/2ab...,https://www.zara.com/de/de/eye-gloss-p24870201...
5648,5648,parfums,damen,EYE GLOSS,von Zara,eye,393335942,"9,95",https://static.zara.net/assets/public/53d0/214...,https://www.zara.com/de/de/eye-gloss-p24870201...
5649,5649,parfums,damen,Creme-Rouge - ZARA FIRST FLUSH CREAM BLUSH - ...,No Color,creme,377447164,"12,95",https://static.zara.net/assets/public/b730/07f...,https://www.zara.com/de/de/creme-rouge---zara-...
5650,5650,parfums,damen,CORRECTOR - Cool - 962 von Zara,No Color,corrector,380732644,"9,95",https://static.zara.net/assets/public/8dcd/48d...,https://www.zara.com/de/de/corrector-p24130142...


In [15]:
df2.isna().sum()

Unnamed: 0      0
Category        0
Sex             0
Product         0
Color           0
Subcategory    65
Reference       0
Price           0
Image           0
Link            0
dtype: int64

In [19]:
df2.dtypes

Unnamed: 0      int64
Category       object
Sex            object
Product        object
Color          object
Subcategory    object
Reference       int64
Price          object
Image          object
Link           object
dtype: object

In [30]:
df2['Price'].unique()

array(['89,95', '69,95', '119,00', '99,95', '59,95', '599,00', '49,95',
       '79,95', '39,95', '149,00', '29,95', '299,00', '229,00', '45,95',
       '139,00', '219,00', '55,95', '65,95', '239,00', '25,95', '199,00',
       '32,95', '179,00', '22,95', '19,95', '17,95', '15,95', '12,95',
       '7,95', '9,95', '8,95', '6,95', '5,95', '159,00', '399,00',
       '129,00', '269,00', '279,00', '35,95'], dtype=object)

In [26]:
df2 = df2.dropna(subset=['Price'])

In [29]:
df2= df2[df2['Price'] != ""]

In [31]:
df2['Price'] = df2['Price'].str.replace(',', '.').astype(float)

In [34]:
df_zara_cleaned = df2.drop(columns=['Unnamed: 0'])

In [41]:
df_zara_cleaned.isnull().sum()

Category        0
Sex             0
Product         0
Color           0
Subcategory    15
Reference       0
Price           0
Image           0
Link            0
dtype: int64

In [42]:
df_zara_cleaned = df_zara_cleaned.dropna(subset=['Subcategory'])

In [43]:
df_zara_cleaned.isnull().sum()

Category       0
Sex            0
Product        0
Color          0
Subcategory    0
Reference      0
Price          0
Image          0
Link           0
dtype: int64

In [44]:
df_zara_cleaned.shape

(5556, 9)

In [47]:
df_zara_cleaned["Color"].unique()

array(['Braun / Braungrau von Zara', 'Grau meliert von Zara',
       'Schwarz von Zara', 'Hellbraun von Zara', 'Dunkelgrau von Zara',
       'Cremefarben von Zara', 'Pastellrosa von Zara',
       'Dunkelbraun von Zara', 'Leopardenprint von Zara', 'Grau von Zara',
       'Nerzfarben von Zara', 'Schwarz / Cremefarben von Zara',
       'Braun von Zara', 'Cremefarben / Braun von Zara', 'Beige von Zara',
       'Khaki von Zara', 'Marineblau von Zara', 'Helles Khaki von Zara',
       'Sandfarben von Zara', 'Beige meliert von Zara',
       'Moosgrün von Zara', 'Blau meliert von Zara',
       'Kamel farben von Zara', 'Hellbeige von Zara', 'Graugrün von Zara',
       'No Color', 'Rot von Zara', 'Senfgelb von Zara',
       'Dunkles Khaki von Zara', 'Toffeefarben von Zara',
       'Goldfarben von Zara', 'Dunkles Indigoblau von Zara',
       'Mittleres Camel von Zara', 'Grün von Zara',
       'Schwarz / Weiß von Zara', 'Schwarz / Silber von Zara',
       'Blau von Zara', 'Mintgrün von Zara', 'Grau

In [48]:
color_mapping = {
    'Braun / Braungrau von Zara': 'Braun/Braungrau',
    'Grau meliert von Zara': 'Meliertes Grau',
    'Schwarz von Zara': 'Schwarz',
    'Hellbraun von Zara': 'Hellbraun',
    'Dunkelgrau von Zara': 'Dunkelgrau',
    'Cremefarben von Zara': 'Cremefarben',
    'Pastellrosa von Zara': 'Pastellrosa',
    'Dunkelbraun von Zara': 'Dunkelbraun',
    'Leopardenprint von Zara': 'Leopardenprint',
    'Grau von Zara': 'Grau',
    'Nerzfarben von Zara': 'Nerzfarben',
    'Schwarz / Cremefarben von Zara': 'Schwarz/Cremefarben',
    'Braun von Zara': 'Braun',
    'Cremefarben / Braun von Zara': 'Cremefarben/Braun',
    'Beige von Zara': 'Beige',
    'Khaki von Zara': 'Khaki',
    'Marineblau von Zara': 'Marineblau',
    'Helles Khaki von Zara': 'Helles Khaki',
    'Sandfarben von Zara': 'Sandfarben',
    'Beige meliert von Zara': 'Meliertes Beige',
    'Moosgrün von Zara': 'Moosgrün',
    'Blau meliert von Zara': 'Meliertes Blau',
    'Kamel farben von Zara': 'Kamelfarben',
    'Hellbeige von Zara': 'Hellbeige',
    'Graugrün von Zara': 'Graugrün',
    'No Color': 'Keine Farbe',
    'Rot von Zara': 'Rot',
    'Senfgelb von Zara': 'Senfgelb',
    'Dunkles Khaki von Zara': 'Dunkles Khaki',
    'Toffeefarben von Zara': 'Toffeefarben',
    'Goldfarben von Zara': 'Goldfarben',
    'Dunkles Indigoblau von Zara': 'Dunkles Indigoblau',
    'Mittleres Camel von Zara': 'Mittleres Camel',
    'Grün von Zara': 'Grün',
    'Schwarz / Weiß von Zara': 'Schwarz/Weiß',
    'Schwarz / Silber von Zara': 'Schwarz/Silber',
    'Blau von Zara': 'Blau',
    'Mintgrün von Zara': 'Mintgrün',
    'Grau / Beige von Zara': 'Grau/Beige',
    'Taupegrau von Zara': 'Taupegrau',
    'Karamell von Zara': 'Karamell',
    'Leopardenmuster / Grau von Zara': 'Leopardenmuster/Grau',
    'Cremefarben / meliert von Zara': 'Cremefarben/Meliert',
    'Perlgrau von Zara': 'Perlgrau',
    'Dunkles Olivengrün von Zara': 'Dunkles Olivengrün',
    'Rot / Schwarz von Zara': 'Rot/Schwarz',
    'DUNKLES KHAKI von Zara': 'Dunkles Khaki',
    'Flieder zart von Zara': 'Zartes Flieder',
    'Einfarbig von Zara': 'Einfarbig',
    'Weiß von Zara': 'Weiß',
    'Cremefarben / Schwarz von Zara': 'Cremefarben/Schwarz',
    'Weiß / Schwarz von Zara': 'Weiß/Schwarz',
    'Weinrot von Zara': 'Weinrot',
    'Mittelblau von Zara': 'Mittelblau',
    'Hellgrau von Zara': 'Hellgrau',
    'Olivgrün von Zara': 'Olivgrün',
    'Hellblau von Zara': 'Hellblau',
    'Steingrau von Zara': 'Steingrau',
    'Indigoblau von Zara': 'Indigoblau',
    'Aubergine von Zara': 'Aubergine',
    'Blau verwaschen von Zara': 'Verwaschenes Blau',
    'Rosa von Zara': 'Rosa',
    'grau-beige von Zara': 'Grau-Beige',
    'Rot / Orange von Zara': 'Rot/Orange',
    'Burgundrot von Zara': 'Burgundrot',
    'Granatrot von Zara': 'Granatrot',
    'Gestreift von Zara': 'Gestreift',
    'Petrol verwaschen von Zara': 'Verwaschenes Petrol',
    'Orange von Zara': 'Orange',
    'Kräftiges Grün von Zara': 'Kräftiges Grün',
    'Pastellgelb von Zara': 'Pastellgelb',
    'Himmelblau von Zara': 'Himmelblau',
    'Weiß / Blau von Zara': 'Weiß/Blau',
    'Hellgrün von Zara': 'Hellgrün',
    'Schlangenmuster von Zara': 'Schlangenmuster',
    'Bunt von Zara': 'Bunt',
    'KAMEL/WEISS von Zara': 'Kamel/Weiß',
    'Olivengrün von Zara': 'Olivengrün',
    'Graphit von Zara': 'Graphit',
    'Blaugrau von Zara': 'Blaugrau',
    'Violett von Zara': 'Violett',
    'Braun/Weiss von Zara': 'Braun/Weiß',
    'Rosa / Weiß von Zara': 'Rosa/Weiß',
    'Silber von Zara': 'Silber',
    'Dunkelgrün von Zara': 'Dunkelgrün',
    'Cava von Zara': 'Cava',
    'Mittelgrau von Zara': 'Mittelgrau',
    'Altrosa von Zara': 'Altrosa',
    'Eis von Zara': 'Eis',
    'Beige / Marineblau von Zara': 'Beige/Marineblau',
    'Braun meliert von Zara': 'Meliertes Braun',
    'Weiß / Rot von Zara': 'Weiß/Rot',
    'Anthrazit dunkel von Zara': 'Dunkles Anthrazit',
    'Helles Petrol von Zara': 'Helles Petrol',
    'Leuchtend Rot von Zara': 'Leuchtend Rot',
    'Cremefarben / Rot von Zara': 'Cremefarben/Rot',
    'Gelb von Zara': 'Gelb',
    'Ocker von Zara': 'Ocker',
    'Blau / Grau von Zara': 'Blau/Grau',
    'Cremefarben / Marineblau von Zara': 'Cremefarben/Marineblau',
    'Meeresgrün von Zara': 'Meeresgrün',
    'Anthrazit von Zara': 'Anthrazit',
    'Verwaschenes Grün von Zara': 'Verwaschenes Grün',
    'Nachtblau von Zara': 'Nachtblau',
    'Tiefblau von Zara': 'Tiefblau',
    'Cremefarben / Beige von Zara': 'Cremefarben/Beige',
    'Dunkelviolett von Zara': 'Dunkelviolett',
    'Dunkelrosa von Zara': 'Dunkelrosa',
    'MARSALA von Zara': 'Marsala',
    'Flieder pudrig von Zara': 'Pudriges Flieder',
    'GRÜN von Zara': 'Grün',
    'Blassblau von Zara': 'Blassblau',
    'Elfenbein farben von Zara': 'Elfenbeinfarben',
    'Sekt farben von Zara': 'Sektfarben',
    'Cremefarben / Grün von Zara': 'Cremefarben/Grün',
    'Braunblau von Zara': 'Braunblau',
    'Hellrosa von Zara': 'Hellrosa',
    'Gold von Zara': 'Gold',
    'Seegrün von Zara': 'Seegrün',
    'Mittelgrün von Zara': 'Mittelgrün',
    'Marineblau / Weiß von Zara': 'Marineblau/Weiß',
    'Lila von Zara': 'Lila',
    'Kobaltblau von Zara': 'Kobaltblau',
    'Marine / Cremefarben von Zara': 'Marine/Cremefarben',
    'Dunkles Bordeauxrot von Zara': 'Dunkles Bordeauxrot',
    'Petrol von Zara': 'Petrol',
    'Mittelrosa von Zara': 'Mittelrosa',
    'Tabakbraun von Zara': 'Tabakbraun',
    'Cremefarben / Blau von Zara': 'Cremefarben/Blau',
    'Dunkles Rostbraun von Zara': 'Dunkles Rostbraun',
    'GRÜNLICH von Zara': 'Grünlich',
    'Leopardenmuster / Rostbraun von Zara': 'Leopardenmuster/Rostbraun',
    'Mittelbeige von Zara': 'Mittelbeige',
    'Print von Zara': 'Print',
    'Rosa pudrig von Zara': 'Pudriges Rosa',
    'Zebra von Zara': 'Zebra',
    'Ziegelrot von Zara': 'Ziegelrot',
    'Bordeaux hell von Zara': 'Helles Bordeauxrot',
    'Bronzefarben von Zara': 'Bronzefarben',
    'Himbeerrot von Zara': 'Himbeerrot',
    'Dunkles Taupe von Zara': 'Dunkles Taupe',
    'Bleigrau von Zara': 'Bleigrau',
    'Limettengrün von Zara': 'Limettengrün',
    'Pink von Zara': 'Pink',
    'Dunkelbeige von Zara': 'Dunkelbeige',
    'Off-White von Zara': 'Off-White',
    'Terrakotta von Zara': 'Terrakotta',
    'Dunkelorange von Zara': 'Dunkelorange',
    'Schwarzgold von Zara': 'Schwarzgold',
    'Dunkles Nerzbraun von Zara': 'Dunkles Nerzbraun',
    'Blau / Indigo von Zara': 'Blau/Indigo',
    'Koralle von Zara': 'Koralle',
    'Helles Rostbraun von Zara': 'Helles Rostbraun',
    'Kupferfarben von Zara': 'Kupferfarben',
    'Aschgrau von Zara': 'Aschgrau',
    'Grün / Cremefarben von Zara': 'Grün/Cremefarben',
    'Braungrau von Zara': 'Braungrau',
    'Grünlich von Zara': 'Grünlich',
    'Sandfarben / Schwarz von Zara': 'Sandfarben/Schwarz',
    'Leinenfarben von Zara': 'Leinenfarben',
    'Grün / Grasgrün von Zara': 'Grün/Grasgrün',
    'Beige / Grün von Zara': 'Beige/Grün',
    'FROSTED GRAY von Zara': 'Frostgrau',
    'Cremefarben / Khaki von Zara': 'Cremefarben/Khaki',
    'Dunkles Aubergine von Zara': 'Dunkles Aubergine',
    'Vanille von Zara': 'Vanille',
    'Ölfarben von Zara': 'Ölfarben',
    'Korallenrosa von Zara': 'Korallenrosa',
    'Pfirsichfarben von Zara': 'Pfirsichfarben',
    'Rötlich von Zara': 'Rötlich',
    'Flieder von Zara': 'Flieder',
    'Helles Türkis von Zara': 'Helles Türkis',
    'Zitronengelb von Zara': 'Zitronengelb',
    'Bläulich von Zara': 'Bläulich',
    'Hellrot von Zara': 'Hellrot',
    'Cremefarben hell von Zara': 'Helles Cremefarben',
    'Wasserblau von Zara': 'Wasserblau',
    'Dunkles Granatrot von Zara': 'Dunkles Granatrot',
    'Nerzfarben metallic von Zara': 'Nerzfarben Metallic',
    'Rot / Weiß von Zara': 'Rot/Weiß',
    'Pastellblau von Zara': 'Pastellblau',
    'Cremefarben / Granatrot von Zara': 'Cremefarben/Granatrot',
    'LIME FIZZ von Zara': 'Lime Fizz',
    'Blau / Grün von Zara': 'Blau/Grün',
    'Dunkles Marineblau von Zara': 'Dunkles Marineblau',
    'Grau Weinrot von Zara': 'Grau/Weinrot',
    'Anthrazitgrau von Zara': 'Anthrazitgrau',
    'Grün meliert von Zara': 'Meliertes Grün',
    'Dunkelrot von Zara': 'Dunkelrot',
    'SAND/BRAUN von Zara': 'Sand/Braun',
    'Khaki meliert von Zara': 'Meliertes Khaki',
    'Gepard von Zara': 'Gepard',
    'Hellgelb von Zara': 'Hellgelb',
    'Weiss von Zara': 'Weiß',
    'Dunkles Camel von Zara': 'Dunkles Camel',
    'Marineblau meliert von Zara': 'Meliertes Marineblau',
    'Grau / Blau von Zara': 'Grau/Blau',
    'Sandfarben / meliert von Zara': 'Meliertes Sandfarben',
    'Grün/Weiß von Zara': 'Grün/Weiß',
    'Perlgrau Vigoré von Zara': 'Perlgrau Vigoré',
    'Mittleres Nerzbraun von Zara': 'Mittleres Nerzbraun',
    'Verwaschenes Rosa von Zara': 'Verwaschenes Rosa',
    'Apfelgrün von Zara': 'Apfelgrün',
    'Flaschengrün von Zara': 'Flaschengrün',
    'Nerzfarben Vigoré von Zara': 'Nerzfarben Vigoré',
    'Zementgrau von Zara': 'Zementgrau',
    'Rot / Koralle von Zara': 'Rot/Koralle',
    'Weiß / Grau von Zara': 'Weiß/Grau',
    'Rot gestreift von Zara': 'Rot Gestreift',
    'Hell Nerzfarben von Zara': 'Helles Nerzfarben',
    'Altgold von Zara': 'Altgold',
    'Pistaziengrün von Zara': 'Pistaziengrün',
    'Weiß / Grün von Zara': 'Weiß/Grün',
    'Dunkles Bleigrau von Zara': 'Dunkles Bleigrau',
    'Weiß / Marineblau von Zara': 'Weiß/Marineblau',
    'Türkis von Zara': 'Türkis',
    'Curry von Zara': 'Curry',
    'Khakigrün von Zara': 'Khakigrün',
    'Leopardenmuster von Zara': 'Leopardenmuster',
    'Gebrochenes Weiß von Zara': 'Gebrochenes Weiß',
    'Mehrfarbig von Zara': 'Mehrfarbig',
    'Bordeauxrot von Zara': 'Bordeauxrot',
    'Sandfarbe von Zara': 'Sandfarbe',
    'Schlangenhautmuster von Zara': 'Schlangenhautmuster',
    'Vanillegelb von Zara': 'Vanillegelb',
    'Maulwurfsgrau von Zara': 'Maulwurfsgrau',
    'Erdbeerrot von Zara': 'Erdbeerrot',
    'Zebramuster von Zara': 'Zebramuster',
    'Rosttöne von Zara': 'Rosttöne',
    'Cremeweiß von Zara': 'Cremeweiß',
    'Transparent von Zara': 'Transparent',
    'Schokoladenbraun von Zara': 'Schokoladenbraun',
    'Silberfarben von Zara': 'Silberfarben',
    'Lachsfarben von Zara': 'Lachsfarben',
    'Jeansblau von Zara': 'Jeansblau',
    'Karamel von Zara': 'Karamell',
    'Tonfarbe von Zara': 'Tonfarbe',
    'Tan von Zara': 'Tan',
    'Apricot von Zara': 'Apricot',
}


In [49]:
# Apply the color mapping to the 'Color' column
df_zara_cleaned['Color'] = df_zara_cleaned['Color'].map(color_mapping)

In [52]:
df_zara_cleaned = df_zara_cleaned.dropna(subset=['Color'])

In [53]:
df_zara_cleaned.isna().sum()

Category       0
Sex            0
Product        0
Color          0
Subcategory    0
Reference      0
Price          0
Image          0
Link           0
dtype: int64

In [54]:
#df_zara_cleaned.to_csv("../01_data/df_zara_cleaned.csv", index=False)

In [75]:
df_zara_cleaned['Color'].unique()

array(['Braun/Braungrau', 'Meliertes Grau', 'Schwarz', 'Hellbraun',
       'Dunkelgrau', 'Cremefarben', 'Pastellrosa', 'Dunkelbraun',
       'Leopardenprint', 'Grau', 'Nerzfarben', 'Schwarz/Cremefarben',
       'Braun', 'Cremefarben/Braun', 'Beige', 'Khaki', 'Marineblau',
       'Helles Khaki', 'Sandfarben', 'Meliertes Beige', 'Moosgrün',
       'Meliertes Blau', 'Kamelfarben', 'Hellbeige', 'Graugrün',
       'Keine Farbe', 'Rot', 'Senfgelb', 'Dunkles Khaki', 'Toffeefarben',
       'Goldfarben', 'Dunkles Indigoblau', 'Mittleres Camel', 'Grün',
       'Schwarz/Weiß', 'Schwarz/Silber', 'Blau', 'Mintgrün', 'Grau/Beige',
       'Taupegrau', 'Karamell', 'Leopardenmuster/Grau',
       'Cremefarben/Meliert', 'Perlgrau', 'Dunkles Olivengrün',
       'Rot/Schwarz', 'Zartes Flieder', 'Einfarbig', 'Weiß',
       'Cremefarben/Schwarz', 'Weiß/Schwarz', 'Weinrot', 'Mittelblau',
       'Hellgrau', 'Olivgrün', 'Hellblau', 'Steingrau', 'Indigoblau',
       'Aubergine', 'Verwaschenes Blau', 'Rosa', 'Gra

In [76]:
main_color_mapping = {
    'Braun/Braungrau': 'Braun',
    'Meliertes Grau': 'Grau',
    'Schwarz': 'Schwarz',
    'Hellbraun': 'Braun',
    'Dunkelgrau': 'Grau',
    'Cremefarben': 'Creme',
    'Pastellrosa': 'Rosa',
    'Dunkelbraun': 'Braun',
    'Leopardenprint': 'Leopardenmuster',
    'Grau': 'Grau',
    'Nerzfarben': 'Braun',
    'Schwarz/Cremefarben': 'Schwarz',
    'Braun': 'Braun',
    'Cremefarben/Braun': 'Braun',
    'Beige': 'Beige',
    'Khaki': 'Grün',
    'Marineblau': 'Blau',
    'Helles Khaki': 'Grün',
    'Sandfarben': 'Beige',
    'Meliertes Beige': 'Beige',
    'Moosgrün': 'Grün',
    'Meliertes Blau': 'Blau',
    'Kamelfarben': 'Braun',
    'Hellbeige': 'Beige',
    'Graugrün': 'Grün',
    'Keine Farbe': 'Keine Farbe',
    'Rot': 'Rot',
    'Senfgelb': 'Gelb',
    'Dunkles Khaki': 'Grün',
    'Toffeefarben': 'Braun',
    'Goldfarben': 'Gold',
    'Dunkles Indigoblau': 'Blau',
    'Mittleres Camel': 'Braun',
    'Grün': 'Grün',
    'Schwarz/Weiß': 'Schwarz',
    'Schwarz/Silber': 'Schwarz',
    'Blau': 'Blau',
    'Mintgrün': 'Grün',
    'Grau/Beige': 'Grau',
    'Taupegrau': 'Grau',
    'Karamell': 'Braun',
    'Leopardenmuster/Grau': 'Leopardenmuster',
    'Cremefarben/Meliert': 'Creme',
    'Perlgrau': 'Grau',
    'Dunkles Olivengrün': 'Grün',
    'Rot/Schwarz': 'Rot',
    'Zartes Flieder': 'Flieder',
    'Einfarbig': 'Einfarbig',
    'Weiß': 'Weiß',
    'Cremefarben/Schwarz': 'Schwarz',
    'Weiß/Schwarz': 'Weiß',
    'Weinrot': 'Rot',
    'Mittelblau': 'Blau',
    'Hellgrau': 'Grau',
    'Olivgrün': 'Grün',
    'Hellblau': 'Blau',
    'Steingrau': 'Grau',
    'Indigoblau': 'Blau',
    'Aubergine': 'Lila',
    'Verwaschenes Blau': 'Blau',
    'Rosa': 'Rosa',
    'Grau-Beige': 'Grau',
    'Rot/Orange': 'Rot',
    'Burgundrot': 'Rot',
    'Granatrot': 'Rot',
    'Gestreift': 'Gestreift',
    'Verwaschenes Petrol': 'Blau',
    'Orange': 'Orange',
    'Kräftiges Grün': 'Grün',
    'Pastellgelb': 'Gelb',
    'Himmelblau': 'Blau',
    'Weiß/Blau': 'Blau',
    'Hellgrün': 'Grün',
    'Schlangenmuster': 'Schlangenmuster',
    'Bunt': 'Bunt',
    'Olivengrün': 'Grün',
    'Graphit': 'Grau',
    'Blaugrau': 'Grau',
    'Violett': 'Lila',
    'Rosa/Weiß': 'Rosa',
    'Silber': 'Silber',
    'Dunkelgrün': 'Grün',
    'Cava': 'Beige',
    'Mittelgrau': 'Grau',
    'Altrosa': 'Rosa',
    'Eis': 'Blau',
    'Beige/Marineblau': 'Blau',
    'Meliertes Braun': 'Braun',
    'Weiß/Rot': 'Weiß',
    'Dunkles Anthrazit': 'Grau',
    'Helles Petrol': 'Blau',
    'Leuchtend Rot': 'Rot',
    'Cremefarben/Rot': 'Rot',
    'Gelb': 'Gelb',
    'Ocker': 'Gelb',
    'Blau/Grau': 'Blau',
    'Cremefarben/Marineblau': 'Blau',
    'Meeresgrün': 'Grün',
    'Anthrazit': 'Grau',
    'Nachtblau': 'Blau',
    'Tiefblau': 'Blau',
    'Cremefarben/Beige': 'Beige',
    'Dunkelviolett': 'Lila',
    'Dunkelrosa': 'Rosa',
    'Marsala': 'Rot',
    'Pudriges Flieder': 'Flieder',
    'Blassblau': 'Blau',
    'Elfenbeinfarben': 'Weiß',
    'Sektfarben': 'Gold',
    'Cremefarben/Grün': 'Grün',
    'Braunblau': 'Braun',
    'Gold': 'Gold',
    'Seegrün': 'Grün',
    'Mittelgrün': 'Grün',
    'Marineblau/Weiß': 'Blau',
    'Lila': 'Lila',
    'Kobaltblau': 'Blau',
    'Marine/Cremefarben': 'Blau',
    'Dunkles Bordeauxrot': 'Rot',
    'Petrol': 'Blau',
    'Mittelrosa': 'Rosa',
    'Tabakbraun': 'Braun',
    'Cremefarben/Blau': 'Blau',
    'Dunkles Rostbraun': 'Braun',
    'Grünlich': 'Grün',
    'Hellrosa': 'Rosa',
    'Leopardenmuster/Rostbraun': 'Leopardenmuster',
    'Mittelbeige': 'Beige',
    'Print': 'Print',
    'Pudriges Rosa': 'Rosa',
    'Zebra': 'Zebramuster',
    'Ziegelrot': 'Rot',
    'Helles Bordeauxrot': 'Rot',
    'Bronzefarben': 'Bronze',
    'Himbeerrot': 'Rot',
    'Dunkles Taupe': 'Braun',
    'Bleigrau': 'Grau',
    'Limettengrün': 'Grün',
    'Pink': 'Pink',
    'Dunkelbeige': 'Beige',
    'Terrakotta': 'Orange',
    'Dunkelorange': 'Orange',
    'Schwarzgold': 'Schwarz',
    'Dunkles Nerzbraun': 'Braun',
    'Blau/Indigo': 'Blau',
    'Koralle': 'Koralle',
    'Helles Rostbraun': 'Braun',
    'Kupferfarben': 'Kupfer',
    'Aschgrau': 'Grau',
    'Grün/Cremefarben': 'Grün',
    'Braungrau': 'Grau',
    'Sandfarben/Schwarz': 'Schwarz',
    'Leinenfarben': 'Beige',
    'Grün/Grasgrün': 'Grün',
    'Beige/Grün': 'Grün',
    'Frostgrau': 'Grau',
    'Cremefarben/Khaki': 'Grün',
    'Dunkles Aubergine': 'Lila',
    'Vanille': 'Gelb',
    'Ölfarben': 'Grün',
    'Korallenrosa': 'Koralle',
    'Pfirsichfarben': 'Orange',
    'Rötlich': 'Rot',
    'Flieder': 'Flieder',
    'Helles Türkis': 'Blau',
    'Zitronengelb': 'Gelb',
    'Bläulich': 'Blau',
    'Hellrot': 'Rot',
    'Helles Cremefarben': 'Creme',
    'Wasserblau': 'Blau',
    'Dunkles Granatrot': 'Rot',
    'Nerzfarben Metallic': 'Braun',
    'Rot/Weiß': 'Rot',
    'Pastellblau': 'Blau',
    'Cremefarben/Granatrot': 'Rot',
    'Lime Fizz': 'Gelb',
    'Blau/Grün': 'Blau',
    'Dunkles Marineblau': 'Blau',
    'Grau/Weinrot': 'Grau',
    'Anthrazitgrau': 'Grau',
    'Meliertes Grün': 'Grün',
    'Dunkelrot': 'Rot',
    'Sand/Braun': 'Braun',
    'Meliertes Khaki': 'Grün',
    'Gepard': 'Gepard',
    'Hellgelb': 'Gelb',
    'Dunkles Camel': 'Braun',
    'Meliertes Marineblau': 'Blau',
    'Grau/Blau': 'Blau',
    'Meliertes Sandfarben': 'Beige',
    'Grün/Weiß': 'Grün',
    'Perlgrau Vigoré': 'Grau',
    'Mittleres Nerzbraun': 'Braun',
    'Apfelgrün': 'Grün',
    'Flaschengrün': 'Grün',
    'Nerzfarben Vigoré': 'Braun',
    'Zementgrau': 'Grau',
    'Rot/Koralle': 'Rot',
    'Weiß/Grau': 'Grau',
    'Rot Gestreift': 'Rot',
    'Helles Nerzfarben': 'Braun',
    'Altgold': 'Gold',
    'Pistaziengrün': 'Grün',
    'Weiß/Grün': 'Grün',
    'Dunkles Bleigrau': 'Grau',
    'Weiß/Marineblau': 'Blau',
    'Türkis': 'Blau',
    'Curry': 'Gelb',
    'Khakigrün': 'Grün',
    'Leopardenmuster': 'Leopardenmuster',
    'Gebrochenes Weiß': 'Weiß',
    'Mehrfarbig': 'Bunt',
    'Bordeauxrot': 'Rot',
    'Sandfarbe': 'Beige',
    'Schlangenhautmuster': 'Schlangenmuster',
    'Vanillegelb': 'Gelb',
    'Maulwurfsgrau': 'Grau',
    'Erdbeerrot': 'Rot',
    'Zebramuster': 'Zebramuster',
    'Rosttöne': 'Orange',
    'Cremeweiß': 'Weiß',
    'Transparent': 'Transparent',
    'Schokoladenbraun': 'Braun',
    'Silberfarben': 'Silber',
    'Lachsfarben': 'Orange',
    'Jeansblau': 'Blau',
    'Tonfarbe': 'Beige',
    'Tan': 'Beige',
    'Apricot': 'Orange'
}


In [77]:
# Apply the color mapping to the 'Color' column
df_zara_cleaned['Color'] = df_zara_cleaned['Color'].map(main_color_mapping)

In [78]:
df_zara_cleaned['Color'].unique()

array(['Braun', 'Grau', 'Schwarz', 'Creme', 'Rosa', 'Leopardenmuster',
       'Beige', 'Grün', 'Blau', 'Keine Farbe', 'Rot', 'Gelb', 'Gold',
       'Flieder', 'Einfarbig', 'Weiß', 'Lila', 'Gestreift', 'Orange',
       'Schlangenmuster', 'Bunt', 'Silber', 'Print', 'Zebramuster',
       'Bronze', 'Pink', 'Koralle', 'Kupfer', 'Gepard', 'Transparent'],
      dtype=object)

In [80]:
df_zara_cleaned['Category'].unique()

array(['maentel', 'jacken', 'blazer', 'kleider', 'hosen', 't_shirts',
       'roecke', 'schuhe', 'taschen', 'parfums'], dtype=object)

In [81]:
#df_zara_cleaned.to_csv("../01_data/df_zara_cleaned_colors.csv", index=False)

In [82]:
df_zara_cleaned

,Category,Sex,Product,Color,Subcategory,Reference,Price,Image,Link
0,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun,langer,410888372,89.95,https://static.zara.net/assets/public/a9e5/f05...,https://www.zara.com/de/de/langer-wendemantel-...
1,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun,langer,410774601,89.95,https://static.zara.net/assets/public/0a65/b2f...,https://www.zara.com/de/de/langer-wendemantel-...
2,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun,langer,410888372,89.95,https://static.zara.net/assets/public/fe16/c55...,https://www.zara.com/de/de/langer-wendemantel-...
3,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun,langer,410888372,89.95,https://static.zara.net/assets/public/868d/7ce...,https://www.zara.com/de/de/langer-wendemantel-...
4,maentel,damen,STRICKMANTEL MIT AUFGESETZTEN TASCHEN,Grau,strickmantel,385398845,69.95,https://static.zara.net/assets/public/0aa6/a50...,https://www.zara.com/de/de/strickmantel-mit-au...
...,...,...,...,...,...,...,...,...,...
5630,parfums,damen,FLÜSSIGER LIDSCHATTEN - Iridiscent Dawn - 257...,Keine Farbe,flussiger,380738612,12.95,https://static.zara.net/assets/public/5ebb/995...,https://www.zara.com/de/de/flussiger-lidschatt...
5634,parfums,damen,No Name,Keine Farbe,zara,380984924,12.95,No Image,https://www.zara.com/de/de/zara-all-that-glimm...
5649,parfums,damen,Creme-Rouge - ZARA FIRST FLUSH CREAM BLUSH - ...,Keine Farbe,creme,377447164,12.95,https://static.zara.net/assets/public/b730/07f...,https://www.zara.com/de/de/creme-rouge---zara-...
5650,parfums,damen,CORRECTOR - Cool - 962 von Zara,Keine Farbe,corrector,380732644,9.95,https://static.zara.net/assets/public/8dcd/48d...,https://www.zara.com/de/de/corrector-p24130142...


In [83]:
# Create Table 1: Categories 
df_categories = pd.DataFrame(df_zara_cleaned['Category'].unique(), columns=['Category'])
df_categories['Category_ID'] = df_categories.index + 1

# Create Table 2: Colors 
df_colors = pd.DataFrame(df_zara_cleaned['Color'].unique(), columns=['Color'])
df_colors['Color_ID'] = df_colors.index + 1

# Create Table 3: Genders (Sex) 
df_genders = pd.DataFrame(df_zara_cleaned['Sex'].unique(), columns=['Gender'])
df_genders['Gender_ID'] = df_genders.index + 1

# Create Table 4: Images with url and link
df_images = df_zara_cleaned[['Image', 'Link']].drop_duplicates().reset_index(drop=True)
df_images['Image_ID'] = df_images.index + 1



In [85]:
# Create Table 5: Products 
df_products = df_zara_cleaned[['Product', 'Reference', 'Price']].drop_duplicates().reset_index(drop=True)
df_products['Product_ID'] = df_products.index + 1

In [87]:
df_products['Image']= df_zara_cleaned["Image"]

In [88]:
df_products['Link']= df_zara_cleaned["Link"]

In [89]:
df_products

,Product,Reference,Price,Product_ID,Image,Link
0,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,410888372,89.95,1,https://static.zara.net/assets/public/a9e5/f05...,https://www.zara.com/de/de/langer-wendemantel-...
1,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,410774601,89.95,2,https://static.zara.net/assets/public/0a65/b2f...,https://www.zara.com/de/de/langer-wendemantel-...
2,STRICKMANTEL MIT AUFGESETZTEN TASCHEN,385398845,69.95,3,https://static.zara.net/assets/public/fe16/c55...,https://www.zara.com/de/de/langer-wendemantel-...
3,KURZER WOLLMANTEL MIT SCHAL,387663993,69.95,4,https://static.zara.net/assets/public/868d/7ce...,https://www.zara.com/de/de/langer-wendemantel-...
4,LANGER MANTEL AUS WOLLMISCHGEWEBE MIT GÜRTEL,409941616,119.00,5,https://static.zara.net/assets/public/0aa6/a50...,https://www.zara.com/de/de/strickmantel-mit-au...
...,...,...,...,...,...,...
5024,FLÜSSIGER LIDSCHATTEN - Iridiscent Dawn - 257...,380988307,12.95,5025,NaN,NaN
5025,FLÜSSIGER LIDSCHATTEN - Iridiscent Dawn - 257...,380738612,12.95,5026,NaN,NaN
5026,No Name,380984924,12.95,5027,https://static.zara.net/assets/public/3ccf/918...,https://www.zara.com/de/de/langes-kleid-mit-fu...
5027,CORRECTOR - Cool - 962 von Zara,380732644,9.95,5028,https://static.zara.net/assets/public/5c2b/3cc...,https://www.zara.com/de/de/kurzer-rock-mit-sch...


In [111]:
#del df_with_ids
#del id_table

In [121]:
# Merge the category table
df_merged = df_zara_cleaned.merge(df_categories, on='Category', how='left')

# Merge the color table
df_merged = df_merged.merge(df_colors, on='Color', how='left')

# Merge the gender table
df_genders = df_genders.rename(columns={'Gender': 'Sex'})
df_merged = df_merged.merge(df_genders, on='Sex', how='left')

# Merge the image table
#df_merged = df_merged.merge(df_images, on=['Image', 'Link'], how='left')
# Merge the products table
df_merged = df_merged.merge(df_products, on=['Product', 'Reference', 'Price'], how='left')

# Final merged dataframe now contains Category_ID, Color_ID, Gender_ID, Image_ID, and Product_ID
df_merged

,Category,Sex,Product,Color,Subcategory,Reference,Price,Image_x,Link_x,Category_ID,Color_ID,Gender_ID,Product_ID,Image_y,Link_y
0,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun,langer,410888372,89.95,https://static.zara.net/assets/public/a9e5/f05...,https://www.zara.com/de/de/langer-wendemantel-...,1,1,1,1,https://static.zara.net/assets/public/a9e5/f05...,https://www.zara.com/de/de/langer-wendemantel-...
1,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun,langer,410774601,89.95,https://static.zara.net/assets/public/0a65/b2f...,https://www.zara.com/de/de/langer-wendemantel-...,1,1,1,2,https://static.zara.net/assets/public/0a65/b2f...,https://www.zara.com/de/de/langer-wendemantel-...
2,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun,langer,410888372,89.95,https://static.zara.net/assets/public/fe16/c55...,https://www.zara.com/de/de/langer-wendemantel-...,1,1,1,1,https://static.zara.net/assets/public/a9e5/f05...,https://www.zara.com/de/de/langer-wendemantel-...
3,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun,langer,410888372,89.95,https://static.zara.net/assets/public/868d/7ce...,https://www.zara.com/de/de/langer-wendemantel-...,1,1,1,1,https://static.zara.net/assets/public/a9e5/f05...,https://www.zara.com/de/de/langer-wendemantel-...
4,maentel,damen,STRICKMANTEL MIT AUFGESETZTEN TASCHEN,Grau,strickmantel,385398845,69.95,https://static.zara.net/assets/public/0aa6/a50...,https://www.zara.com/de/de/strickmantel-mit-au...,1,2,1,3,https://static.zara.net/assets/public/fe16/c55...,https://www.zara.com/de/de/langer-wendemantel-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5187,parfums,damen,FLÜSSIGER LIDSCHATTEN - Iridiscent Dawn - 257...,Keine Farbe,flussiger,380738612,12.95,https://static.zara.net/assets/public/5ebb/995...,https://www.zara.com/de/de/flussiger-lidschatt...,10,10,1,5026,NaN,NaN
5188,parfums,damen,No Name,Keine Farbe,zara,380984924,12.95,No Image,https://www.zara.com/de/de/zara-all-that-glimm...,10,10,1,5027,https://static.zara.net/assets/public/3ccf/918...,https://www.zara.com/de/de/langes-kleid-mit-fu...
5189,parfums,damen,Creme-Rouge - ZARA FIRST FLUSH CREAM BLUSH - ...,Keine Farbe,creme,377447164,12.95,https://static.zara.net/assets/public/b730/07f...,https://www.zara.com/de/de/creme-rouge---zara-...,10,10,1,4997,NaN,NaN
5190,parfums,damen,CORRECTOR - Cool - 962 von Zara,Keine Farbe,corrector,380732644,9.95,https://static.zara.net/assets/public/8dcd/48d...,https://www.zara.com/de/de/corrector-p24130142...,10,10,1,5028,https://static.zara.net/assets/public/5c2b/3cc...,https://www.zara.com/de/de/kurzer-rock-mit-sch...


In [123]:
df_merged.drop_duplicates()

,Category,Sex,Product,Color,Subcategory,Reference,Price,Image_x,Link_x,Category_ID,Color_ID,Gender_ID,Product_ID,Image_y,Link_y
0,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun,langer,410888372,89.95,https://static.zara.net/assets/public/a9e5/f05...,https://www.zara.com/de/de/langer-wendemantel-...,1,1,1,1,https://static.zara.net/assets/public/a9e5/f05...,https://www.zara.com/de/de/langer-wendemantel-...
1,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun,langer,410774601,89.95,https://static.zara.net/assets/public/0a65/b2f...,https://www.zara.com/de/de/langer-wendemantel-...,1,1,1,2,https://static.zara.net/assets/public/0a65/b2f...,https://www.zara.com/de/de/langer-wendemantel-...
2,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun,langer,410888372,89.95,https://static.zara.net/assets/public/fe16/c55...,https://www.zara.com/de/de/langer-wendemantel-...,1,1,1,1,https://static.zara.net/assets/public/a9e5/f05...,https://www.zara.com/de/de/langer-wendemantel-...
3,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun,langer,410888372,89.95,https://static.zara.net/assets/public/868d/7ce...,https://www.zara.com/de/de/langer-wendemantel-...,1,1,1,1,https://static.zara.net/assets/public/a9e5/f05...,https://www.zara.com/de/de/langer-wendemantel-...
4,maentel,damen,STRICKMANTEL MIT AUFGESETZTEN TASCHEN,Grau,strickmantel,385398845,69.95,https://static.zara.net/assets/public/0aa6/a50...,https://www.zara.com/de/de/strickmantel-mit-au...,1,2,1,3,https://static.zara.net/assets/public/fe16/c55...,https://www.zara.com/de/de/langer-wendemantel-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5187,parfums,damen,FLÜSSIGER LIDSCHATTEN - Iridiscent Dawn - 257...,Keine Farbe,flussiger,380738612,12.95,https://static.zara.net/assets/public/5ebb/995...,https://www.zara.com/de/de/flussiger-lidschatt...,10,10,1,5026,NaN,NaN
5188,parfums,damen,No Name,Keine Farbe,zara,380984924,12.95,No Image,https://www.zara.com/de/de/zara-all-that-glimm...,10,10,1,5027,https://static.zara.net/assets/public/3ccf/918...,https://www.zara.com/de/de/langes-kleid-mit-fu...
5189,parfums,damen,Creme-Rouge - ZARA FIRST FLUSH CREAM BLUSH - ...,Keine Farbe,creme,377447164,12.95,https://static.zara.net/assets/public/b730/07f...,https://www.zara.com/de/de/creme-rouge---zara-...,10,10,1,4997,NaN,NaN
5190,parfums,damen,CORRECTOR - Cool - 962 von Zara,Keine Farbe,corrector,380732644,9.95,https://static.zara.net/assets/public/8dcd/48d...,https://www.zara.com/de/de/corrector-p24130142...,10,10,1,5028,https://static.zara.net/assets/public/5c2b/3cc...,https://www.zara.com/de/de/kurzer-rock-mit-sch...


In [128]:
df_merged1 = df_merged.drop_duplicates(subset=['Link_x', 'Image_x']).reset_index(drop=True)

In [129]:
df_merged1

,Category,Sex,Product,Color,Subcategory,Reference,Price,Image_x,Link_x,Category_ID,Color_ID,Gender_ID,Product_ID,Image_y,Link_y
0,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun,langer,410888372,89.95,https://static.zara.net/assets/public/a9e5/f05...,https://www.zara.com/de/de/langer-wendemantel-...,1,1,1,1,https://static.zara.net/assets/public/a9e5/f05...,https://www.zara.com/de/de/langer-wendemantel-...
1,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun,langer,410774601,89.95,https://static.zara.net/assets/public/0a65/b2f...,https://www.zara.com/de/de/langer-wendemantel-...,1,1,1,2,https://static.zara.net/assets/public/0a65/b2f...,https://www.zara.com/de/de/langer-wendemantel-...
2,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun,langer,410888372,89.95,https://static.zara.net/assets/public/fe16/c55...,https://www.zara.com/de/de/langer-wendemantel-...,1,1,1,1,https://static.zara.net/assets/public/a9e5/f05...,https://www.zara.com/de/de/langer-wendemantel-...
3,maentel,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun,langer,410888372,89.95,https://static.zara.net/assets/public/868d/7ce...,https://www.zara.com/de/de/langer-wendemantel-...,1,1,1,1,https://static.zara.net/assets/public/a9e5/f05...,https://www.zara.com/de/de/langer-wendemantel-...
4,maentel,damen,STRICKMANTEL MIT AUFGESETZTEN TASCHEN,Grau,strickmantel,385398845,69.95,https://static.zara.net/assets/public/0aa6/a50...,https://www.zara.com/de/de/strickmantel-mit-au...,1,2,1,3,https://static.zara.net/assets/public/fe16/c55...,https://www.zara.com/de/de/langer-wendemantel-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5026,parfums,damen,FLÜSSIGER LIDSCHATTEN - Iridiscent Dawn - 257...,Keine Farbe,flussiger,380738612,12.95,https://static.zara.net/assets/public/5ebb/995...,https://www.zara.com/de/de/flussiger-lidschatt...,10,10,1,5026,NaN,NaN
5027,parfums,damen,No Name,Keine Farbe,zara,380984924,12.95,No Image,https://www.zara.com/de/de/zara-all-that-glimm...,10,10,1,5027,https://static.zara.net/assets/public/3ccf/918...,https://www.zara.com/de/de/langes-kleid-mit-fu...
5028,parfums,damen,Creme-Rouge - ZARA FIRST FLUSH CREAM BLUSH - ...,Keine Farbe,creme,377447164,12.95,https://static.zara.net/assets/public/b730/07f...,https://www.zara.com/de/de/creme-rouge---zara-...,10,10,1,4997,NaN,NaN
5029,parfums,damen,CORRECTOR - Cool - 962 von Zara,Keine Farbe,corrector,380732644,9.95,https://static.zara.net/assets/public/8dcd/48d...,https://www.zara.com/de/de/corrector-p24130142...,10,10,1,5028,https://static.zara.net/assets/public/5c2b/3cc...,https://www.zara.com/de/de/kurzer-rock-mit-sch...


In [126]:
df_merged["Product_ID"].nunique()

5029

In [130]:
df_merged1 = df_merged.drop_duplicates(subset=['Link_y', 'Image_y']).reset_index(drop=True)

In [132]:
# Select only the columns with IDs
df_ids = df_merged1[['Category_ID', 'Color_ID', 'Gender_ID', 'Product_ID']].drop_duplicates()

In [133]:
df_ids

,Category_ID,Color_ID,Gender_ID,Product_ID
0,1,1,1,1
1,1,1,1,2
2,1,2,1,3
3,1,3,1,4
4,1,1,1,5
...,...,...,...,...
4814,10,10,1,5021
4815,10,10,1,5022
4816,10,10,1,5027
4817,10,10,1,5028


### SQL

In [90]:
from sqlalchemy import create_engine
import pymysql
import getpass


password = getpass.getpass('Please enter your database password: ')
connection_string = 'mysql+pymysql://root:' + password + '@localhost/zara'
engine = create_engine(connection_string)

In [92]:
df_products.to_sql('products', con=engine, if_exists='replace', index=False)

5029

In [93]:
df_categories.to_sql('category', con=engine, if_exists='replace', index=False)

10

In [94]:
df_colors.to_sql('color', con=engine, if_exists='replace', index=False)

30

In [96]:
df_genders.to_sql('gender', con=engine, if_exists='replace', index=False)

1

In [135]:
df_ids.to_sql('ids', con=engine, if_exists='replace', index=False)

4819